In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.layered.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.lqns.*
import jline.solvers.ln.*

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.SILENT)

println("This example illustrates the execution on a layered queueing network model.")
println("Performance indexes now refer to processors, tasks, entries, and activities.")
println("Indexes refer to the submodel (layer) where the processor or task acts as a server.")
println("NaN indexes indicate that the metric is not supported by the node type.")

// Create model
val model = LayeredNetwork("myLayeredModel")

In [ ]:
// Create processors
val P = mutableMapOf<Int, Processor>()
P[1] = Processor(model, "P1", 1, SchedStrategy.PS)
P[2] = Processor(model, "P2", 1, SchedStrategy.PS)

// Create tasks
val T = mutableMapOf<Int, Task>()
T[1] = Task(model, "T1", 10, SchedStrategy.REF).on(P[1]!!).setThinkTime(Exp.fitMean(100.0))
T[2] = Task(model, "T2", 1, SchedStrategy.FCFS).on(P[2]!!).setThinkTime(Immediate())

// Create entries
val E = mutableMapOf<Int, Entry>()
E[1] = Entry(model, "E1").on(T[1]!!)
E[2] = Entry(model, "E2").on(T[2]!!)

In [ ]:
// Create activities
val A = mutableMapOf<Int, Activity>()
A[1] = Activity(model, "AS1", Exp.fitMean(1.6)).on(T[1]!!).boundTo(E[1]!!)
A[2] = Activity(model, "AS2", Immediate()).on(T[1]!!).synchCall(E[2]!!, 1.0)
A[3] = Activity(model, "AS3", Exp.fitMean(5.0)).on(T[2]!!).boundTo(E[2]!!)
A[4] = Activity(model, "AS4", Exp.fitMean(1.0)).on(T[2]!!).repliesTo(E[2]!!)

// Add precedences
T[1]!!.addPrecedence(ActivityPrecedence.Serial(A[1]!!, A[2]!!))
T[2]!!.addPrecedence(ActivityPrecedence.Serial(A[3]!!, A[4]!!))

In [ ]:
// Set options
val options = LQNS.defaultOptions()
options.keep = true // Keep intermediate XML files

// Solve with LQNS
val solvers = mutableMapOf<Int, NetworkSolver>()
val avgTables = mutableMapOf<Int, Any>()

if (LQNS.isAvailable()) {
    solvers[1] = LQNS(model)
    try {
        avgTables[1] = solvers[1]!!.avgTable
        println("AvgTable[1] (LQNS):")
        (avgTables[1] as AvgTable).print()
    } catch (e: Exception) {
        println("LQNS solver error: ${e.message}")
    }
} else {
    println("LQNS solver not available")
}

In [ ]:
// Get raw tables if available
if (LQNS.isAvailable() && solvers[1] != null) {
    try {
        // Note: getRawAvgTables might not be available in Kotlin version
        println("Raw tables method not yet implemented in Kotlin version")
    } catch (e: Exception) {
        println("getRawAvgTables method not yet implemented: ${e.message}")
    }
}

// Solve with LN
try {
    avgTables[3] = LN(model).avgTable
    println("AvgTable[3] (LN):")
    (avgTables[3] as AvgTable).print()
} catch (e: Exception) {
    println("LN solver error: ${e.message}")
}